In [ ]:
            case MagicType.SwiftBlade:
            case MagicType.SeismicSlam://天雷锤
                return MirAnimation.Combat3;

In [ ]:
public virtual void SetFrame(ObjectAction action);
{
    Direction = action.Direction;
    CurrentLocation = action.Location;
    ...
    switch (action.Action)
    {
     case MirAction.Mining:
         MiningEffect = (bool)action.Extra[0];
         break;
     ...
     case MirAction.Spell:
         MagicType = (MagicType)action.Extra[0];
         //目标ID列表(群法)
         targets = (List<uint>)action.Extra[1];
         //目标对象列表
         AttackTargets = new List<MapObject>();
         ...
         switch (MagicType)
         {
         ...
         case MagicType.SeismicSlam://天雷锤
         Effects.Add(spell = new MirEffect(4900, 6, TimeSpan.FromMilliseconds(100), LibraryFile.MagicEx5, 10, 35, Globals.LightningColour)
         {
             Blend = true,
             Target = this,
             Magic = MagicType.SeismicSlam,
             Direction = action.Direction,
         });
         break;
         ....
    }
}

In [ ]:
public virtual void FrameIndexChanged()
{
    switch (CurrentAction)
    {
    case MirAction.Attack:
        if (FrameIndex != 1) return;
        PlayAttackSound();
        break;
        ....

    case MirAction.Spell:
        switch (MagicType)
        {
        case MagicType.SeismicSlam:   //天雷锤
            if (FrameIndex == 4)
            {
                Effects.Add(new MirEffect(700, 7, TimeSpan.FromMilliseconds(120), LibraryFile.MonMagicEx7, 10, 35, Globals.LightningColour)
                {
                    Blend = true,
                    Magic = MagicType.SeismicSlam,
                    MapTarget = Functions.Move(CurrentLocation, Direction, 2),
                });
                //震动屏幕计数
                //User.ShakeScreenCount = 10;
            }break;

In [ ]:
public virtual void Draw()
{
    if (DeathHide && Target != null && Target.Dead) return;

    if (CEnvir.Now < StartTime || Library == null) return;

    float opacity = Config.MagicOpacity ? Config.MagicOpacityValue : Opacity;
    float bopacity = Config.MagicOpacity ? Config.MagicOpacityValue : BlendRate;
    switch(Magic)
    {
    // 无法关闭的效果
    case MagicType.MagicShield:
    case MagicType.DragonRise:
    case MagicType.CelestialLight:
    case MagicType.FlamingSword:
    case MagicType.DestructiveSurge:
    case MagicType.HalfMoon:
    case MagicType.StrengthChop:
    case MagicType.Purification:
    case MagicType.BladeStorm:
    case MagicType.MoltenShield:
    case MagicType.SeismicSlam: //天雷锤
        {
            opacity = Opacity;
            bopacity = BlendRate;
        }break;
    default:
        if(MapFlag)
        {
            opacity = Opacity;
            bopacity = BlendRate;
        }
        break;
    }
    if (Blend)
        Library.DrawBlend(DrawFrame, DrawX, DrawY, DrawColour, UseOffSet, bopacity , ImageType.Image);
    else
        Library.Draw(DrawFrame, DrawX, DrawY, DrawColour, UseOffSet, opacity , ImageType.Image);
}

In [ ]:
public bool UseMagic(ClientUserMagic magic)
{
    MirDirection direction = Game.MapControl.MouseDirection();

    switch (magic.Info.Magic)
    {
    case MagicType.ShoulderDash://野蛮冲撞
        if (CEnvir.Now < ServerTime) return false;
        if ((Poison & PoisonType.WraithGrip) == PoisonType.WraithGrip) return false;
    
        ServerTime = CEnvir.Now.AddSeconds(5);
        NextMagicTime = CEnvir.Now + Globals.MagicDelay;
        CEnvir.Enqueue(new C.Magic { Direction = direction, Action = MirAction.Spell, Type = magic.Info.Magic });
        return true;
    ....
    case MagicType.SeismicSlam://天雷锤  
    case MagicType.Repulsion://抗拒火环
    case MagicType.ScortchedEarth://地狱火
    case MagicType.LightningBeam://极光电影
    case MagicType.Teleportation://瞬息移动
    case MagicType.FrozenEarth://冰沙掌
    case MagicType.BlowEarth://风震天
    case MagicType.GreaterFrozenEarth://冰破刺
    case MagicType.ThunderStrike://电闪雷鸣
    case MagicType.MirrorImage://分身术
    case MagicType.Invisibility://隐身术
    case MagicType.TaoistCombatKick://空拳刀法
    case MagicType.ThunderKick:  //横扫千军
    case MagicType.SummonSkeleton://召唤术
    case MagicType.SummonShinsu://召唤神兽
    case MagicType.SummonJinSkeleton://超强召唤骷髅
    case MagicType.SummonDemonicCreature://炎魔召唤术
    case MagicType.DemonExplosion://魔焰强解术
    case MagicType.Scarecrow://迷魂大法
    case MagicType.PoisonousCloud://毒云
    case MagicType.Cloak://潜行
    case MagicType.SummonPuppet:  //亡灵替身
    case MagicType.TheNewBeginning://心机一转
    case MagicType.DarkConversion://黄泉旅者
    case MagicType.DragonRepulse://狂涛泉涌
    case MagicType.FlashOfLight://日闪
    case MagicType.Evasion://风之闪避
    case MagicType.RagingWind://风之守护
    case MagicType.SwiftBlade://快刀斩马
    case MagicType.Transparency://秒影无踪
        break;//不检测技能范围 

In [ ]:
foreach (MirEffect ob in Effects)
{
    bool show = false;
    switch (ob.Magic)
    {
    case MagicType.SeismicSlam:
        show = true;//天雷锤特效始终显示
        break;
    }
    if(ob.EffectType == EffectType.ItemLighting)
    {
        show = true;
    }
    if (!show && !ob.NoHide) continue;

In [ ]:
public void Attack(MapObject ob, List<UserMagic> magics, bool primary, int extra)  //攻击（地图对象，技能，限制，附加值）
{
    if (ob?.Node == null || ob.Dead) return;
    
    ....
    
    int power = GetDC();          //攻击力
    int karmaDamage = 0;          //伤害值
    bool ignoreAccuracy = false, hasFlameSplash = false, hasLotus = false, hasDestructiveSurge = false,hasHalfMoon = false;   //忽略准确  新月炎龙 莲花 十方斩 半月
    bool hasBladeStorm = false; //莲月
    bool hasDanceOfSallows = false; //  鹰击 
    bool hasMassacre = false;                             //最后抵抗
    bool hasSwiftBlade = false, hasSeismicSlam = false;   //旋风斩 天雷锤
    //bool hasStrengthChop = false;
    UserMagic magic;
    foreach (UserMagic mag in magics)
    {
        switch (mag.Info.Magic)
        {
        case MagicType.FullBloom:         //盛开
        case MagicType.WhiteLotus:        //白莲
        case MagicType.RedLotus:          //红莲
        ....
        case MagicType.SwiftBlade:       //快刀斩马
        case MagicType.SeismicSlam:      //天雷锤
            ignoreAccuracy = true;       //忽略准确
            hasSwiftBlade = true;        //快刀斩马
            break;
        ....
        }
    }
    ....
    int accuracy = Stats[Stat.Accuracy];   //准确

    int res;
    
    if (!ignoreAccuracy && SEnvir.Random.Next(ob.Stats[Stat.Agility]) > accuracy)   //忽视  随机准确 > 准确
    {
        ob.Dodged();    //躲避
        return;
    }
    bool hasStone = Equipment[(int)EquipmentSlot.Amulet]?.Info.ItemType == ItemType.DarkStone;   //护身符 或 暗石
    int weaponDamage = SEnvir.Random.Next(2) + 1;
    for (int i = magics.Count - 1; i >= 0; i--)
    {
        magic = magics[i];
        int bonus;
        switch (magic.Info.Magic)
        {
        case MagicType.Throw:
            power += magic.GetPower();
            weaponDamage = 1000;//减飞镖持久
            break;
        case MagicType.Slaying:                 //攻杀
        ....
        case MagicType.SeismicSlam:         //天雷锤
            power = (int)(power * magic.GetPower() * 1.000 / 100);
            power += (int)(power * (Stats [ Stat.SeismicSlamHoist ] * 1.000 / 100));
            //if (ob.Race == ObjectType.Player)
            //    power /= 2;                 //打人伤害减半
        
            hasSeismicSlam = true;
        
            break;
        ....
        }
    }
    
    ....
        
    if ((primary || Class == MirClass.Warrior || hasFlameSplash) && lifestealAmount > 1)   //技能限制  或 战士 或  新月炎龙爆
        LifeSteal += lifestealAmount;
    
    if (LifeSteal > 1 && !hasSeismicSlam)      //吸血 天雷锤不能吸血
    {
        int heal = (int)Math.Floor(LifeSteal);
        LifeSteal -= heal;
        ChangeHP(Math.Min((hasLotus ? 1500 : 750), heal));   //如果是定义 最大1500  否则最大值750
                                                             //DisplayLifeSteal = true;
    }
    
    int psnRate = 200;
    
    if (ob.Level >= 250)
        psnRate = 2000;
    
    //天雷锤去掉麻痹
    if (SEnvir.Random.Next(psnRate) < Stats[Stat.ParalysisChance] /*|| hasSeismicSlam*/)
    {
        ob.ApplyPoison(new Poison
        {
            Owner = this,
            Type = PoisonType.Paralysis,
            TickFrequency = TimeSpan.FromSeconds(3),
            TickCount = 1,
        });
    }
    
    //天雷锤去掉石化
    //if (hasSeismicSlam)
    //{
    //    ob.ApplyPoison(new Poison
    //    {
    //        Type = PoisonType.WraithGrip,
    //        Owner = this,
    //        TickCount = 1,
    //        TickFrequency = TimeSpan.FromMilliseconds(1500),
    //    });
    //}
    
    if (ob.Race != ObjectType.Player && SEnvir.Random.Next(psnRate) < Stats[Stat.SlowChance])
    {
        ob.ApplyPoison(new Poison
        {
            Owner = this,
            Type = PoisonType.Slow,
            Value = 20,
            TickFrequency = TimeSpan.FromSeconds(5),
            TickCount = 1,
        });
    }
    
    //天雷锤去掉沉默
    if (SEnvir.Random.Next(psnRate) < Stats[Stat.SilenceChance] /*|| hasSeismicSlam*/)
    {
        ob.ApplyPoison(new Poison
        {
            Owner = this,
            Type = PoisonType.Silenced,
            TickFrequency = TimeSpan.FromSeconds(5),
            TickCount = 1,
        });
    }
    
    foreach (UserMagic mag in magics)
        LevelMagic(mag);
    ....
}

In [ ]:
 public override void ProcessAction(DelayedAction action)  //过程操作
 {
     switch (action.Type)
     {
     case ActionType.DelayAttack:
         Attack((MapObject)action.Data[0], (int)action.Data[1], (Element)action.Data[2]);
         return;
     case ActionType.DelayMagic:
         switch ((MagicType)action.Data[0])
         {
         case MagicType.FireWall:
             FireWallEnd((Cell)action.Data[1]);
             break;
             ....
         case MagicType.SeismicSlam:
             SeismicSlamEnd((Cell)action.Data[1]);
             break;
             ....
         }
         ....
     }
     ....
}


public void SeismicSlam(int distance=3)
{
    if (Target != null) Direction = Functions.DirectionFromPoint(CurrentLocation, Target.CurrentLocation);
    
    Point location = Functions.Move(CurrentLocation, Direction, 2);

    List<uint> targetIDs = new List<uint>();
    List<Point> locations = new List<Point>();

    Broadcast(new S.ObjectMagic { ObjectID = ObjectID, Direction = Direction, CurrentLocation = CurrentLocation, Cast = true, Type = MagicType.SeismicSlam, Targets = targetIDs, Locations = locations });

    List<Cell> cells = CurrentMap.GetCells(location, 0, distance);
    foreach (Cell cell in cells)
    {
        if (cell.Objects == null)
        {
            locations.Add(cell.Location);
            continue;
        }
        foreach (MapObject ob in cell.Objects)
        {
            if (!CanAttackTarget(ob)) continue;

            targetIDs.Add(ob.ObjectID);

            ActionList.Add(new DelayedAction(
                SEnvir.Now.AddMilliseconds(500),
                ActionType.DelayAttack,
                ob,
                GetMC(),
                Element.Lightning));
        }
        ActionList.Add(new DelayedAction(SEnvir.Now.AddMilliseconds(500),ActionType.DelayMagic,MagicType.SeismicSlam,cell));
    }
}

public void SeismicSlamEnd(Cell cell)
{
    List<uint> targetIDs = new List<uint>();
    
    if (cell == null || cell.Objects == null) return;

    foreach (MapObject ob in cell.Objects)
    {
        if (!CanAttackTarget(ob)) continue;

        //麻痹
        ob.ApplyPoison(new Poison
        {
            Owner = this,
            Type = PoisonType.Paralysis,
            TickFrequency = TimeSpan.FromSeconds(3),
            TickCount = 1,
        });

        //幽灵石化
        ob.ApplyPoison(new Poison
        {
            Type = PoisonType.WraithGrip,
            Owner = this,
            TickCount = 1,
            TickFrequency = TimeSpan.FromMilliseconds(1500),
        });
        //沉默
        ob.ApplyPoison(new Poison
        {
            Owner = this,
            Type = PoisonType.Silenced,
            TickFrequency = TimeSpan.FromSeconds(5),
            TickCount = 1,
        });
    }
}

In [ ]:
public void Magic(C.Magic p)  //技能
{
    ....
        
    if (!CanCast)
    {
       Enqueue(new S.UserLocation { Direction = Direction, Location = CurrentLocation });
        return;
    }
    switch (p.Type)
    {
    case MagicType.Throw:
    case MagicType.ShoulderDash:
    case MagicType.Interchange:
    case MagicType.Defiance:
    case MagicType.Beckon:
    case MagicType.Might:
    case MagicType.ReflectDamage:
    case MagicType.Fetter:
    case MagicType.SwiftBlade:
    case MagicType.Endurance:
    case MagicType.Assault:
    ....
    case MagicType.SeismicSlam: //天雷锤
    if (magic.Cost > CurrentMP)
    {
        Enqueue(new S.UserLocation { Direction = Direction, Location = CurrentLocation });
        return;
    }
    break;
    ....
    bool cast = true;
    List<uint> targets = new List<uint>();
    List<Point> locations = new List<Point>();
    
    List<Cell> cells;
    Stats stats;
    int power;
    UserMagic augMagic;
    HashSet<MapObject> realTargets;
    List<UserMagic> magics;
    int _shape;
    
    int count;
    List<MapObject> possibleTargets;
    Point location;
    BuffInfo buff;
    switch (p.Type)
    {
    #region Warrior
    //战士
    case MagicType.ShoulderDash://野蛮冲撞
        ....
    case MagicType.SeismicSlam://天雷锤
        ob = null;
    
        cells = CurrentMap.GetCells( Functions.Move( CurrentLocation , p.Direction , 2 ) , 0 , 1 );
        SwiftBladeLifeSteal = 0;
    
        foreach (Cell cell in cells)
        {
            ActionList.Add( new DelayedAction(
                SEnvir.Now.AddMilliseconds( 600 ) ,
                ActionType.DelayMagic ,
                new List<UserMagic> { magic } ,
                cell ) );
        }
        break;
    ....
    

In [ ]:
public void CompleteMagic(params object[] data)
{
    List<UserMagic> magics = (List<UserMagic>)data[0];
    foreach (UserMagic magic in magics)
    {

        switch (magic.Info.Magic)
        {

        #region Warrior
        case MagicType.Interchange:
            InterchangeEnd(magic, (MapObject)data[1]);
            break;
            ....
        case MagicType.SwiftBlade:
        case MagicType.SeismicSlam:
            Cell cell = (Cell)data[1];
            if (cell == null || cell.Objects == null) continue;

            for (int i = cell.Objects.Count - 1; i >= 0; i--)
            {
                if (!CanAttackTarget(cell.Objects[i])) continue;
                Attack(cell.Objects[i], magics, true, 0);
            }

            break;
        ....
        }
        ....
    }
    ....
}